# Preprocessing


## Imports

In [1]:
import pandas as pd
import posixpath
from data_mining_project import data, preprocessing, DATA_PATH, OUTPUT_PATH
import numpy as np
import matplotlib as plt
import plotly.express as px

## Load Data

In [2]:
file_name = "sncb_data_challenge.csv"  
file_path = posixpath.join(DATA_PATH, file_name)
data_df = data.load_data_csv(file_path)
data_df.head(5)

,Column1,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
0,0,4432881,"[609, 609, 609, 609, 609, 609, 609, 609, 609, ...","[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...",50.876601,4.718143,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",4
1,1,4432943,"[526, 526, 526, 526, 526, 526, 526, 526, 526, ...","[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...",51.037435,4.431218,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",13
2,2,4432955,"[592, 592, 592, 592, 592, 592, 592, 592, 592, ...","[4394, 1566, 1570, 4114, 4168, 4168, 4156, 406...","[-12291, -12291, -12291, -10932, -10932, -1091...",50.864083,4.162115,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, False, True, Tr...",14
3,3,4433021,"[576, 576, 576, 576, 576, 576, 576, 576, 576, ...","[4066, 4066, 4066, 4066, 4068, 2742, 4026, 270...","[-14351, -14204, -13890, -13383, -12739, -1243...",51.183220,4.276025,"[0.0, 0.0, 0.0, 0.015625, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",2
4,4,4433129,"[634, 634, 634, 634, 634, 634, 634, 634, 634, ...","[4002, 4032, 4028, 2852, 4026, 4110, 2742, 285...","[-224, -224, -223, -222, -222, -222, -220, -22...",50.818727,3.253601,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",14


In [3]:
data_df = data.reformat_str_to_list(data_df, cols=["vehicles_sequence", "events_sequence", "seconds_to_incident_sequence", "dj_ac_state_sequence", "dj_dc_state_sequence"], col_type=int)
data_df = data.reformat_str_to_list(data_df, cols=["train_kph_sequence"], col_type=float)
data_df.drop(columns=["incident_id", "Column1", "vehicles_sequence", "approx_lat", "approx_lon"], inplace=True)
data_df.head(5)

,events_sequence,seconds_to_incident_sequence,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
0,"[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
1,"[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13
2,"[4394, 1566, 1570, 4114, 4168, 4168, 4156, 406...","[-12291, -12291, -12291, -10932, -10932, -1091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",14
3,"[4066, 4066, 4066, 4066, 4068, 2742, 4026, 270...","[-14351, -14204, -13890, -13383, -12739, -1243...","[0.0, 0.0, 0.0, 0.015625, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
4,"[4002, 4032, 4028, 2852, 4026, 4110, 2742, 285...","[-224, -224, -223, -222, -222, -222, -220, -22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",14


## Remove events < 4h before the incident and > 10min after the incident

In [4]:
interval= [-3600*4, 360]
data_df = preprocessing.filter_events_out_of_interval(data_df, interval)
data_df.head(5)

,events_sequence,seconds_to_incident_sequence,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
0,"[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
1,"[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13
2,"[4394, 1566, 1570, 4114, 4168, 4168, 4156, 406...","[-12291, -12291, -12291, -10932, -10932, -1091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",14
3,"[4066, 4066, 4066, 4066, 4068, 2742, 4026, 270...","[-14351, -14204, -13890, -13383, -12739, -1243...","[0.0, 0.0, 0.0, 0.015625, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
4,"[4002, 4032, 4028, 2852, 4026, 4110, 2742, 285...","[-224, -224, -223, -222, -222, -222, -220, -22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",14


## Remove rows shorted than 2 elements

In [5]:
data_df = data_df[data_df["events_sequence"].apply(lambda row: preprocessing.remove_short_rows(row, x=2))].reset_index(drop=True)
data_df.shape

(980, 6)

## AC_DC Failure Feature

In [6]:
def indicate_ac_dc_prob_timestamp(row):
    return (row.iloc[0] == row.iloc[1]).astype(int)

def indicate_ac_dc_prob(row):
    return (row.iloc[0] == row.iloc[1]).any().astype(int)

def indicate_ac_dc_num(row):
    return (row.iloc[0] == row.iloc[1]).sum()

#def indicate_ac_dc_prob_events(row):
#    return row.iloc[2][row.iloc[0] == row.iloc[1]]

data_df.insert(1, "ac_dc_prob_timestamp", data_df[["dj_ac_state_sequence", "dj_dc_state_sequence"]].apply(indicate_ac_dc_prob_timestamp, axis=1))
data_df.insert(1, "ac_dc_prob", data_df[["dj_ac_state_sequence", "dj_dc_state_sequence"]].apply(indicate_ac_dc_prob, axis=1))
data_df.insert(1, "ac_dc_prob_num", data_df[["dj_ac_state_sequence", "dj_dc_state_sequence"]].apply(indicate_ac_dc_num, axis=1))
#data_df.insert(1, "ac_dc_prob_events", data_df[["dj_ac_state_sequence", "dj_dc_state_sequence", "events_sequence"]].apply(indicate_ac_dc_prob_events, axis=1))

data_df

,events_sequence,ac_dc_prob_num,ac_dc_prob,ac_dc_prob_timestamp,seconds_to_incident_sequence,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
0,"[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...",108,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
1,"[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13
2,"[4394, 1566, 1570, 4114, 4168, 4168, 4156, 406...",216,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[-12291, -12291, -12291, -10932, -10932, -1091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",14
3,"[4066, 4066, 4066, 4066, 4068, 2742, 4026, 270...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-14351, -14204, -13890, -13383, -12739, -1243...","[0.0, 0.0, 0.0, 0.015625, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
4,"[4002, 4032, 4028, 2852, 4026, 4110, 2742, 285...",27,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-224, -224, -223, -222, -222, -222, -220, -22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",14
...,...,...,...,...,...,...,...,...,...
975,"[4002, 2852, 4110, 2854, 4026, 4092, 4094, 261...",112,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-13963, -13961, -13961, -13959, -13958, -1395...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",14
976,"[3490, 4068, 4068, 4068, 4068, 4068, 4068, 406...",2,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-14384, -14347, -14229, -14060, -13941, -1383...","[83.2, 0.4, 0.1, 0.2, 0.2, 0.2, 0.1, 0.3, 0.0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
977,"[4066, 4066, 4066, 4068, 3658, 4066, 3658, 406...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-14187, -13803, -13251, -12756, -12643, -1207...","[0.2, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.1, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
978,"[2956, 2956, 2956, 2956, 2956, 2956, 2956, 295...",27,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-14400, -14337, -14335, -14307, -14296, -1427...","[116.4, 97.2, 96.9, 94.4, 95.3, 93.7, 94.3, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",13


## Add hard breaking feature

In [7]:
data_df[['hard_braking','description']] = data_df.apply(preprocessing.detect_braking, axis=1)
data_df.head(5)

,events_sequence,ac_dc_prob_num,ac_dc_prob,ac_dc_prob_timestamp,seconds_to_incident_sequence,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type,hard_braking,description
0,"[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...",108,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,0,[]
1,"[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13,0,[]
2,"[4394, 1566, 1570, 4114, 4168, 4168, 4156, 406...",216,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[-12291, -12291, -12291, -10932, -10932, -1091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",14,0,[]
3,"[4066, 4066, 4066, 4066, 4068, 2742, 4026, 270...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-14351, -14204, -13890, -13383, -12739, -1243...","[0.0, 0.0, 0.0, 0.015625, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0,[]
4,"[4002, 4032, 4028, 2852, 4026, 4110, 2742, 285...",27,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-224, -224, -223, -222, -222, -222, -220, -22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",14,0,[]


## Save data to outputs

In [7]:
filepath = posixpath.join(OUTPUT_PATH, "preprocessed_data.csv")
data.save_data(data_df, filepath)